# Model Context Protocol (MCP) Labs - PART 2 (Google Colab)

## 🚀 Running on Google Colab

**Continuation of Part 1** - Advanced MCP integrations:

### Part 2 Labs:
- **Lab 5**: Discord MCP - Team Communication
- **Lab 6**: GitHub MCP - Code Portfolio Review
- **Lab 7**: Custom MCP Servers - Domain-Specific Tools
- **Lab 8**: Multi-MCP Orchestration - Complete Workflows

### Prerequisites:
✅ Completed Part 1 (Gmail & Calendar setup)

### Use Cases:
1. **HR Recruitment** - Complete hiring workflow
2. **AirBnb Management** - Complete booking workflow

---

# Setup: Quick Start or Full Restore

## Option 1: Continue from Part 1 (Same Session)

If you're running Part 2 immediately after Part 1, **skip to Lab 5**.

All setup is already done!

## Option 2: New Session - Restore from Google Drive

In [ ]:
# If starting fresh session and saved to Drive in Part 1
from google.colab import drive
import os

print("💾 Restore from Google Drive?")
print("   (Only if you saved config in Part 1)\n")

restore = input("Restore from Drive? (yes/no): ").lower() == 'yes'

if restore:
    # Mount Drive
    drive.mount('/content/drive')
    
    # Restore files
    !mkdir -p /content/mcp_config
    !cp -r /content/drive/MyDrive/mcp_labs_config/* /content/mcp_config/ 2>/dev/null || echo "⚠️ No saved config found"
    !cp /content/drive/MyDrive/mcp_labs_config/mcp_config.json /content/ 2>/dev/null || echo "⚠️ No mcp_config.json found"
    
    if os.path.exists('/content/mcp_config/credentials.json'):
        print("\n✅ Configuration restored from Drive")
    else:
        print("\n❌ No saved configuration found. Run Part 1 first.")
else:
    print("\n⏭️ Skipped. Make sure Part 1 setup is complete.")

## Option 3: Fresh Setup - Run Part 1 First

If you haven't run Part 1, go back and complete:
- Google Cloud setup
- OAuth authentication
- MCP servers installation

---

# Core Setup (Run Always)

In [ ]:
# Install/Verify Node.js (if new session)
import os

if os.system("which node > /dev/null 2>&1") != 0:
    print("📦 Installing Node.js...")
    !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
    !sudo apt-get install -y nodejs
    print("✅ Node.js installed")
else:
    print("✅ Node.js already available")

In [ ]:
# Install Python packages
!pip install -q langchain-openai langgraph langchain-core python-dotenv
!pip install -q langchain-mcp-adapters mcp
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q google-api-python-client

print("✅ Python packages installed")

In [ ]:
# Install MCP servers (if new session)
%%bash
if ! npm list -g @modelcontextprotocol/server-gmail > /dev/null 2>&1; then
    echo "📦 Installing MCP servers..."
    npm install -g @modelcontextprotocol/server-gmail
    npm install -g @modelcontextprotocol/server-google-calendar
    npm install -g @modelcontextprotocol/server-filesystem
    npm install -g @modelcontextprotocol/server-github
    echo "✅ MCP servers installed"
else
    echo "✅ MCP servers already installed"
fi

In [ ]:
# Core imports
import os
import json
from pathlib import Path
from getpass import getpass
from datetime import datetime, timedelta

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import MessagesState
from langgraph.prebuilt import create_react_agent

# Set OpenAI API Key
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key from Colab secrets")
except:
    OPENAI_API_KEY = getpass("Enter OpenAI API Key: ")

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Configuration
SENDER_EMAIL = "cygenaidemo@gmail.com"

print("✅ Setup complete")
print(f"📧 Sender: {SENDER_EMAIL}")

In [ ]:
# Sample data
!mkdir -p /content/data/resumes /content/data/contracts

HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "test@example.com",
            "github": "priya-sharma",
            "discord_id": "priya_dev#1234",
            "status": "screening"
        }
    }
}

AIRBNB_DATABASE = {
    "properties": {
        "PROP001": {"name": "Cozy Goa Beach Villa", "price_per_night": 8500}
    },
    "bookings": {
        "BOOK001": {
            "guest_name": "Sarah Johnson",
            "guest_email": "guest@example.com",
            "property_id": "PROP001",
            "status": "confirmed"
        }
    }
}

print("✅ Sample data loaded")

---

# LAB 5: Discord MCP - Team Communication

**Use Cases**:
- HR: Interview coordination, candidate updates
- AirBnb: Guest support, host alerts

**Note**: Discord MCP requires a Discord Bot Token. For this lab, we'll use simulated tools.

## Optional: Discord Bot Setup

To use real Discord:
1. Go to [Discord Developer Portal](https://discord.com/developers/applications)
2. Create New Application → Bot
3. Copy Bot Token
4. Add to Colab Secrets as `DISCORD_BOT_TOKEN`
5. Invite bot to your server

In [ ]:
# Discord MCP Tools (Simulated for demo)

@tool
def send_discord_message(channel_id: str, message: str, mention_user: str = "") -> str:
    """
    Send message to Discord channel.
    
    Args:
        channel_id: Channel name (e.g., 'hr-interviews', 'guest-support')
        message: Message content
        mention_user: User to mention (e.g., '@rahul_verma')
    """
    print(f"""
💬 DISCORD MESSAGE
───────────────────────────────────────
Channel: #{channel_id}
{f'Mention: {mention_user}' if mention_user else ''}
───────────────────────────────────────
{message}
───────────────────────────────────────
""")
    return f"✅ Posted to #{channel_id}"

@tool
def create_discord_thread(channel_id: str, thread_name: str, initial_message: str, members: str = "") -> str:
    """
    Create Discord thread.
    
    Args:
        channel_id: Parent channel
        thread_name: Thread name
        initial_message: First message
        members: Comma-separated usernames
    """
    print(f"""
🧵 DISCORD THREAD
───────────────────────────────────────
Thread: {thread_name}
Channel: #{channel_id}
Members: {members}
───────────────────────────────────────
{initial_message}
───────────────────────────────────────
""")
    return f"✅ Thread '{thread_name}' created"

@tool
def send_discord_dm(user_id: str, message: str) -> str:
    """Send direct message."""
    print(f"📨 DM to {user_id}: {message[:100]}...")
    return f"✅ DM sent to {user_id}"

@tool
def post_discord_embed(channel_id: str, title: str, description: str, color: str = "blue") -> str:
    """
    Post rich embed.
    
    Args:
        channel_id: Channel
        title: Embed title
        description: Description
        color: Color (blue, green, red, yellow)
    """
    colors = {"blue": "🔵", "green": "🟢", "red": "🔴", "yellow": "🟡"}
    print(f"""
📋 DISCORD EMBED
───────────────────────────────────────
Channel: #{channel_id}
{colors.get(color, '⚪')} {title}
───────────────────────────────────────
{description}
───────────────────────────────────────
""")
    return f"✅ Embed posted"

discord_tools = [send_discord_message, create_discord_thread, send_discord_dm, post_discord_embed]
discord_agent = create_react_agent(
    llm,
    tools=discord_tools,
    state_modifier="Discord Communication Assistant. Use embeds for announcements, threads for discussions."
)

print("✅ Discord Agent ready (simulated)")

In [ ]:
# Test Lab 5
print("🧪 LAB 5: Discord MCP\n" + "="*80)

# Test 1: HR Interview Announcement
print("\n--- Test 1: Post Interview Schedule ---")
result1 = discord_agent.invoke({
    "messages": [HumanMessage(content="""
Post interview schedule in #hr-interviews:
- Priya Sharma - Senior Backend - Oct 15, 10:00 AM
- Interviewers: @rahul_verma, @vikram_singh
Use blue embed with details.
""")]
})

# Test 2: AirBnb Guest Support
print("\n--- Test 2: Guest Support Alert ---")
result2 = discord_agent.invoke({
    "messages": [HumanMessage(content="""
Post in #guest-support:
- New booking: Sarah Johnson
- Property: Goa Beach Villa
- Check-in: Oct 20
- Special: Early check-in, veg meals
Alert @ravi_goa
""")]
})

# Test 3: Emergency
print("\n--- Test 3: Emergency Alert ---")
result3 = discord_agent.invoke({
    "messages": [HumanMessage(content="""
URGENT in #host-alerts:
- AC breakdown at Mountain View Cottage
- Guest checked in today
- Temp: 32°C
Mention @maintenance_team
Use RED embed.
""")]
})

print("\n✅ Lab 5 Complete!")

---

# LAB 6: GitHub MCP - Code Portfolio Review

**Use Case**: HR - Evaluate candidate's GitHub profile

## Optional: GitHub Token Setup

For real GitHub API access:
1. Go to [GitHub Settings → Developer settings → Tokens](https://github.com/settings/tokens)
2. Generate classic token with `repo` and `user` scopes
3. Add to Colab Secrets as `GITHUB_TOKEN`

In [ ]:
# GitHub MCP Tools (Simulated)

@tool
def get_github_profile(username: str) -> str:
    """Get GitHub user profile."""
    profiles = {
        "priya-sharma": {
            "username": "priya-sharma",
            "name": "Priya Sharma",
            "bio": "Senior Backend Engineer | Python | AWS",
            "public_repos": 42,
            "followers": 156,
            "contributions_last_year": 847,
            "top_languages": ["Python (65%)", "JavaScript (20%)", "Go (10%)"],
            "notable_repos": [
                "fastapi-microservice-template",
                "aws-lambda-toolkit",
                "python-design-patterns"
            ]
        }
    }
    return json.dumps(profiles.get(username, {"error": "Not found"}), indent=2)

@tool
def get_repository_info(username: str, repo_name: str) -> str:
    """Get repository details."""
    repo = {
        "name": repo_name,
        "description": "Production-ready FastAPI microservice template",
        "stars": 234,
        "forks": 45,
        "language": "Python",
        "topics": ["fastapi", "microservices", "docker", "kubernetes"],
        "has_ci_cd": True,
        "test_coverage": "87%",
        "last_updated": "2025-10-05"
    }
    return json.dumps(repo, indent=2)

@tool
def analyze_code_quality(username: str, repo_name: str) -> str:
    """Analyze code quality."""
    analysis = {
        "overall_quality": "Good",
        "metrics": {
            "maintainability_index": 78,
            "test_coverage": "87%",
            "documentation_score": 9.2
        },
        "strengths": [
            "Well-structured code",
            "Comprehensive tests",
            "Good type hints",
            "Follows PEP 8"
        ],
        "improvements": [
            "Refactor complex functions",
            "Add integration tests"
        ]
    }
    return json.dumps(analysis, indent=2)

@tool
def get_contribution_activity(username: str, days: int = 90) -> str:
    """Get contribution activity."""
    activity = {
        "username": username,
        "period": f"Last {days} days",
        "total_commits": 127,
        "total_prs": 18,
        "avg_commits_per_week": 9.8,
        "consistency": "High - almost daily contributions"
    }
    return json.dumps(activity, indent=2)

github_tools = [get_github_profile, get_repository_info, analyze_code_quality, get_contribution_activity]
github_agent = create_react_agent(
    llm,
    tools=github_tools,
    state_modifier="Technical Portfolio Reviewer. Evaluate GitHub profiles and provide hiring recommendations."
)

print("✅ GitHub Agent ready (simulated)")

In [ ]:
# Test Lab 6
print("🧪 LAB 6: GitHub MCP\n" + "="*80)

print("\n--- Comprehensive GitHub Portfolio Review ---")
result = github_agent.invoke({
    "messages": [HumanMessage(content="""
Perform comprehensive GitHub review for Priya Sharma (priya-sharma):

1. Get profile and activity level
2. Analyze 'fastapi-microservice-template' repository
3. Assess code quality
4. Review contribution consistency (90 days)
5. Provide technical assessment for hiring manager
6. Suggest interview questions based on portfolio
""")]
})

for msg in result['messages']:
    if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
        print(f"\n🤖 {msg.content}")
        break

print("\n✅ Lab 6 Complete!")

---

# LAB 7: Custom MCP Servers - Domain-Specific Tools

In [ ]:
# HR Custom Tools

@tool
def get_candidate(candidate_id: str) -> str:
    """Get candidate from HR database."""
    candidate = HR_DATABASE["candidates"].get(candidate_id)
    return json.dumps(candidate, indent=2) if candidate else f"Not found: {candidate_id}"

@tool
def update_candidate_status(candidate_id: str, new_status: str) -> str:
    """
    Update candidate status.
    
    Args:
        candidate_id: Candidate ID
        new_status: screening, interview_scheduled, offer_sent, hired, rejected
    """
    if candidate_id in HR_DATABASE["candidates"]:
        old = HR_DATABASE["candidates"][candidate_id]["status"]
        HR_DATABASE["candidates"][candidate_id]["status"] = new_status
        name = HR_DATABASE["candidates"][candidate_id]["name"]
        return f"✅ {name}: {old} → {new_status}"
    return f"❌ Not found: {candidate_id}"

# AirBnb Custom Tools

@tool
def get_property(property_id: str) -> str:
    """Get property details."""
    prop = AIRBNB_DATABASE["properties"].get(property_id)
    return json.dumps(prop, indent=2) if prop else f"Not found: {property_id}"

@tool
def get_booking(booking_id: str) -> str:
    """Get booking details."""
    booking = AIRBNB_DATABASE["bookings"].get(booking_id)
    return json.dumps(booking, indent=2) if booking else f"Not found: {booking_id}"

@tool
def update_booking_status(booking_id: str, new_status: str) -> str:
    """
    Update booking status.
    
    Args:
        new_status: pending_payment, confirmed, checked_in, checked_out, cancelled
    """
    if booking_id in AIRBNB_DATABASE["bookings"]:
        old = AIRBNB_DATABASE["bookings"][booking_id]["status"]
        AIRBNB_DATABASE["bookings"][booking_id]["status"] = new_status
        return f"✅ {booking_id}: {old} → {new_status}"
    return f"❌ Not found: {booking_id}"

hr_custom_tools = [get_candidate, update_candidate_status]
airbnb_custom_tools = [get_property, get_booking, update_booking_status]

print("✅ Custom MCP tools created")
print(f"  - HR tools: {len(hr_custom_tools)}")
print(f"  - AirBnb tools: {len(airbnb_custom_tools)}")
print("\n💡 In production: Expose these using to_fastmcp()")

---

# LAB 8: Multi-MCP Orchestration - Complete Workflows

**Combining ALL MCP servers for end-to-end automation**

In [ ]:
# Import tools from Part 1 (Gmail & Calendar)
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from email.mime.text import MIMEText
import base64

# Load credentials
gmail_creds = Credentials.from_authorized_user_file('/content/mcp_config/gmail-token.json')
calendar_creds = Credentials.from_authorized_user_file('/content/mcp_config/calendar-token.json')

gmail_service = build('gmail', 'v1', credentials=gmail_creds)
calendar_service = build('calendar', 'v3', credentials=calendar_creds)

# Gmail tool
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send email via Gmail."""
    try:
        message = MIMEText(body)
        message['to'] = to
        message['from'] = SENDER_EMAIL
        message['subject'] = subject
        
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        result = gmail_service.users().messages().send(userId='me', body={'raw': raw}).execute()
        return f"✅ Email sent: {result['id']}"
    except Exception as e:
        return f"❌ Failed: {str(e)}"

# Calendar tool
@tool
def create_calendar_event(summary: str, start_time: str, end_time: str, attendees: str, location: str = "") -> str:
    """Create calendar event."""
    try:
        event = {
            'summary': summary,
            'location': location,
            'start': {'dateTime': start_time, 'timeZone': 'Asia/Kolkata'},
            'end': {'dateTime': end_time, 'timeZone': 'Asia/Kolkata'},
            'attendees': [{'email': e.strip()} for e in attendees.split(',')]
        }
        result = calendar_service.events().insert(calendarId='primary', body=event).execute()
        return f"✅ Event created: {result['id']}"
    except Exception as e:
        return f"❌ Failed: {str(e)}"

# Filesystem tools
@tool
def read_file(file_path: str) -> str:
    """Read file."""
    if "priya" in file_path.lower():
        return "PRIYA SHARMA - Senior Engineer, 6+ years Python/AWS"
    return "File content..."

print("✅ All tools imported for orchestration")

## HR Complete Workflow Agent

In [ ]:
# Combine ALL HR tools
all_hr_tools = (
    [read_file] +                    # Filesystem
    [send_email] +                   # Gmail (REAL)
    [create_calendar_event] +        # Calendar (REAL)
    discord_tools +                  # Discord
    github_tools +                   # GitHub
    hr_custom_tools                  # Custom HR
)

hr_complete_agent = create_react_agent(
    llm,
    tools=all_hr_tools,
    state_modifier=f"""
You are a Complete HR Recruitment Automation System.

Available tools:
- Filesystem: Resume management
- Gmail: Send emails from {SENDER_EMAIL}
- Calendar: Create real calendar events
- Discord: Team coordination
- GitHub: Technical assessment
- HR Database: Candidate tracking

Execute complete hiring workflows.
"""
)

print(f"✅ HR Complete Agent: {len(all_hr_tools)} tools")

In [ ]:
# Test: Complete HR Workflow
print("🧪 LAB 8A: Complete HR Workflow\n" + "="*80)
print("⚠️ This will send REAL email and create REAL calendar event!\n")

TEST_EMAIL = input("Enter test email for interview invitation: ")

if TEST_EMAIL and "@" in TEST_EMAIL:
    result = hr_complete_agent.invoke({
        "messages": [HumanMessage(content=f"""
Execute COMPLETE hiring workflow for Priya Sharma (CAN001):

STEP 1 - SCREENING:
- Read resume from /content/data/resumes/priya.txt
- Review GitHub profile (priya-sharma)
- Analyze repository: fastapi-microservice-template
- Update candidate status to 'technical_review_complete'

STEP 2 - SCHEDULING:
- Create calendar event:
  * Title: Technical Interview - Priya Sharma
  * Date: 2025-10-15T10:00:00+05:30 to 2025-10-15T11:30:00+05:30
  * Attendees: {TEST_EMAIL}
  * Location: https://meet.google.com/test-interview

STEP 3 - COMMUNICATION:
- Send interview invitation email to {TEST_EMAIL}
- Post announcement in Discord #hr-interviews
- Update status to 'interview_scheduled'

STEP 4 - SUMMARY:
Provide complete workflow summary with GitHub assessment.
""")]
    })
    
    print("\n✅ Workflow Complete! Check email and calendar.")
else:
    print("⏭️ Skipped - No valid email provided")

print("\n✅ Lab 8A Complete!")

## AirBnb Complete Workflow Agent

In [ ]:
# Combine ALL AirBnb tools
all_airbnb_tools = (
    [read_file] +                    # Filesystem
    [send_email] +                   # Gmail (REAL)
    [create_calendar_event] +        # Calendar (REAL)
    discord_tools +                  # Discord
    airbnb_custom_tools              # Custom AirBnb
)

airbnb_complete_agent = create_react_agent(
    llm,
    tools=all_airbnb_tools,
    state_modifier=f"""
You are a Complete AirBnb Property Management System.

Available tools:
- Filesystem: Contracts, documents
- Gmail: Send emails from {SENDER_EMAIL}
- Calendar: Block property dates
- Discord: Host coordination
- Database: Property/booking management

Execute complete guest workflows.
"""
)

print(f"✅ AirBnb Complete Agent: {len(all_airbnb_tools)} tools")

In [ ]:
# Test: Complete AirBnb Workflow
print("🧪 LAB 8B: Complete AirBnb Workflow\n" + "="*80)
print("⚠️ This will send REAL email!\n")

GUEST_EMAIL = input("Enter test email for booking confirmation: ")

if GUEST_EMAIL and "@" in GUEST_EMAIL:
    result = airbnb_complete_agent.invoke({
        "messages": [HumanMessage(content=f"""
Execute COMPLETE booking workflow for Sarah Johnson (BOOK001):

STEP 1 - CONFIRMATION:
- Get booking details (BOOK001)
- Get property details (PROP001)
- Update booking status to 'confirmed'

STEP 2 - GUEST COMMUNICATION:
- Send booking confirmation to {GUEST_EMAIL}:
  * Property: Cozy Goa Beach Villa
  * Total: ₹42,500 (5 nights × ₹8,500)
  * Check-in: Oct 20, 2025 (2 PM)
  * Check-out: Oct 25, 2025 (11 AM)
  * WiFi: GoaBeach2025
  * House rules and amenities

STEP 3 - HOST COORDINATION:
- Post booking in Discord #guest-support
- Alert property manager @ravi_goa

STEP 4 - SUMMARY:
Provide complete workflow summary.
""")]
    })
    
    print("\n✅ Workflow Complete! Check email.")
else:
    print("⏭️ Skipped - No valid email provided")

print("\n✅ Lab 8B Complete!")

---

# 🎉 Part 2 Complete!

## What You Achieved:

✅ **Lab 5**: Discord MCP - Team communication patterns

✅ **Lab 6**: GitHub MCP - Technical portfolio review

✅ **Lab 7**: Custom MCP - Domain-specific tools

✅ **Lab 8**: Multi-MCP Orchestration - Complete workflows

## Complete MCP Stack:

From Part 1 + Part 2:
- ✅ Filesystem MCP
- ✅ Gmail MCP (REAL)
- ✅ Calendar MCP (REAL)
- ✅ Discord MCP (simulated)
- ✅ GitHub MCP (simulated)
- ✅ Custom MCP servers

## Real vs Simulated:

**REAL (Production-ready)**:
- ✅ Gmail API - Sending emails
- ✅ Calendar API - Creating events

**Simulated (Demo)**:
- Discord - Requires bot token
- GitHub - Requires personal token

## Key Learnings:

### 1. MCP Architecture
- Client-server model
- Transport types (stdio)
- Multi-server coordination

### 2. Real API Integration
- OAuth 2.0 flows
- Google APIs in Colab
- Token management

### 3. Agent Patterns
- Tool-enabled agents
- Multi-tool orchestration
- Workflow automation

### 4. Production Deployment
- Error handling
- Credential management
- Colab persistence

## Next Steps:

### To Enable More Real MCPs:

**Discord**:
1. Create Discord bot at discord.com/developers
2. Add `DISCORD_BOT_TOKEN` to Colab secrets
3. Install: `npm install -g discord-mcp-server`
4. Update mcp_config.json

**GitHub**:
1. Generate token at github.com/settings/tokens
2. Add `GITHUB_TOKEN` to Colab secrets
3. MCP server already installed
4. Update mcp_config.json

### Production Deployment:

```python
# For production, use MultiServerMCPClient
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(mcp_config)
tools = client.get_tools()
agent = create_react_agent(llm, tools=tools)
```

## 📚 Resources:

- [MCP Specification](https://modelcontextprotocol.io/)
- [LangChain MCP Docs](https://docs.langchain.com/oss/python/langchain/mcp)
- [MCP Servers Registry](https://mcpservers.org)
- [Google Cloud Console](https://console.cloud.google.com/)

## 🌟 You Built:

**Two Complete Agentic Systems**:
1. HR Recruitment Automation
2. AirBnb Property Management

**With Real Capabilities**:
- Sending actual emails
- Creating real calendar events
- Managing documents
- Coordinating workflows

Congratulations! 🎉 You've mastered MCP in Google Colab! 🚀